# Add an object to the interactive

*This will take a name and use the information from wikipedia to generate an entry*

In [1]:
import pandas as pd
import numpy as np
import os
import json
import wikipedia
import wptools #https://github.com/siznax/wptools

*Info for WWT*

In [2]:
def getWWTcmd(RA,DEC,view):
    return 'http://tilewall.adlerplanetarium.org:5050/layerApi.aspx?cmd=mode&lookat=Sky&flyto='+str(RA)+','+str(DEC)+','+str(view)+'0,0'

*Info from wikipedia*

In [3]:
def getWikiInfo(search, view=4):
    #https://wikipedia.readthedocs.io/en/latest/quickstart.html
    
    site = wikipedia.search(search)
    print(search)
    if (site):
        print("wiki page: ", site[0])
        page = None
        try:
            page = wikipedia.page(site[0])
        except wikipedia.exceptions.DisambiguationError as e:
            print(e.options)
            #pass
        if (page):
            img = []
            for i in page.images:
                if (("jpg" in i or "png" in i) and "Celestia" not in i):
                    img.append(i)
                    
            #get more info with wptools
            #https://github.com/siznax/wptools/wiki/
            wpage = wptools.page(search)
            wpage.get_query()
            wpage.get_parse()
            
            #get the main image
            mainImg = None
            if (wpage.images()):
                for i in wpage.images():
                    if (i['kind'] == 'query-pageimage'):
                        mainImg = i['url']

                if (mainImg):
                    img2 = img
                    img = [mainImg]
                    for i in img2:
                        if (i != mainImg):
                            img.append(i)
            
            #get the infobox for RA, Dec, etc.
            RA = None
            Dec = None
            Distance = None
            Size = None
            wwt = None
            
            info = wpage.data['infobox']
            if ('ra' in info):
                RAstring = info['ra']
                print('RAstring', RAstring)
                p1 = RAstring.find('|')
                RAstring = RAstring[(p1+1):]
                p1 = RAstring.find('|')
                rh = RAstring[:p1]
                RAstring = RAstring[(p1+1):]
                p1 = RAstring.find('|')
                rm = RAstring[:p1]
                RAstring = RAstring[(p1+1):]
                p1 = RAstring.find('}')
                rs = RAstring[:p1]
                print('RA', rh, rm, rs)
                RA = float(rh) + float(rm)/60. + float(rs)/3600.
            if ('dec' in info):
                DecString = info['dec']
                print('DecString', DecString)
                p1 = DecString.find('|')
                DecString = DecString[(p1+1):]
                p1 = DecString.find('|')
                dd = DecString[:p1]
                DecString = DecString[(p1+1):]
                p1 = DecString.find('|')
                dm = DecString[:p1]
                DecString = DecString[(p1+1):]
                p1 = DecString.find('}')
                ds = DecString[:p1]
                print('Dec', dd, dm, ds)
                Dec = float(dd) + float(dm)/60. + float(ds)/3600.
            if ('distance' in info):
                DistString = info['distance']
                print('DistString', DistString)
                p1 = DistString.find('|')
                DistString = DistString[(p1+1):]
                p1 = DistString.find('|')
                d = DistString[:p1]
                DistString = DistString[(p1+1):]
                p1 = DistString.find('|')
                u = DistString[:p1]
                Distance = d + ' ' + u
                print('Distance ', Distance)
            if ('size' in info):
                Size = info['size']
            
            if (RA and Dec and view):
                wwt = getWWTcmd(RA,Dec,view)
            return img, page.url, page.summary, RA, Dec, Distance, Size, wwt

    return None, None, None, None, None, None, None

*Create the json file*

In [4]:
def makeOutput(fname, nme, WWTurl=None, Distance=None, Size=None, Notes=None, Category=None, images=None, wikipedia=None):
    dictOut = {}
    dictOut[nme] = {}
    dictOut[nme]['WWTurl'] = WWTurl
    dictOut[nme]['Distance'] = Distance
    dictOut[nme]['Size'] = Size
    dictOut[nme]['Notes'] = Notes
    dictOut[nme]['Category'] = Category
    dictOut[nme]['images'] = images
    dictOut[nme]['wikipedia'] = wikipedia
    
    with open(fname, 'w') as fp:
        json.dump(dictOut, fp)

### A method to create the object in one line

In [5]:
def createObject(nme, cat, fname):
    img, url, summary, RA, Dec, Distance, Size, wwt = getWikiInfo(nme)
    makeOutput(fname, 
               nme,
               WWTurl=wwt, 
               Distance=Distance, 
               Size=Size, 
               Notes=summary, 
               Category=cat, 
               images=img, 
               wikipedia=url)

## Create a couple new objects

In [6]:
nme = "SN 1987a"
cat = "Nebula"
fname = 'SN1987a.json'
createObject(nme, cat, fname)

SN 1987a
wiki page:  SN 1987A


en.wikipedia.org (query) SN 1987a
en.wikipedia.org (imageinfo) File:Eso0708a.jpg
SN 1987A (en) data
{
  WARNINGS: <dict(1)> extracts
  assessments: <dict(2)> Astronomy, Physics
  description: supernova
  extext: <str(969)> **SN 1987A** was a peculiar type II supernova...
  extract: <str(1033)> <p class="mw-empty-elt"></p><p><b>SN 1987A<...
  image: <list(2)> {'kind': 'query-pageimage', 'file': 'File:Eso07...
  label: SN 1987A
  length: 38,413
  links: <list(187)> Albert F. A. L. Jones, All Sky Automated Surv...
  modified: <dict(1)> page
  pageid: 28930
  random: Jean-Jacques Boisard
  redirected: <list(1)> {'from': 'SN 1987a', 'to': 'SN 1987A'}
  redirects: <list(14)> {'pageid': 192119, 'ns': 0, 'title': 'Supe...
  requests: <list(2)> query, imageinfo
  title: SN 1987A
  url: https://en.wikipedia.org/wiki/SN_1987A
  url_raw: https://en.wikipedia.org/wiki/SN_1987A?action=raw
  watchers: 103
  wikibase: Q584905
  wikidata_url: https://www.wikidata.org/wiki/Q584905
}
en.wikipedia.org (pa

RAstring {{RA|05|35|28.03}}
RA 05 35 28.03
DecString {{DEC|-69|16|11.79}}
Dec -69 16 11.79
DistString {{convert|51.4|kpc|ly|abbr|=|on}}
Distance  51.4 kpc


In [7]:
nme = "Cassiopeia A"
cat = "Nebula"
fname = 'CassiopeiaA.json'
createObject(nme, cat, fname)



Cassiopeia A
wiki page:  Cassiopeia A


en.wikipedia.org (query) Cassiopeia A
en.wikipedia.org (imageinfo) File:Cassiopeia A Spitzer Crop.jpg
Cassiopeia A (en) data
{
  WARNINGS: <dict(1)> extracts
  assessments: <dict(1)> Astronomy
  description: supernova remnant in the constellation Cassiopeia
  extext: <str(1658)> **Cassiopeia A** ( **Cas A** ) is a supernov...
  extract: <str(1714)> <p><b>Cassiopeia A</b> (<b>Cas A</b>) is a ...
  image: <list(2)> {'kind': 'query-pageimage', 'file': 'File:Cassi...
  label: Cassiopeia A
  length: 14,046
  links: <list(140)> 3 Cassiopeiae, Aerobee, All Sky Automated Sur...
  modified: <dict(1)> page
  pageid: 1996903
  random: Burin-Placentia West
  redirects: <list(6)> {'pageid': 1499533, 'ns': 0, 'title': '3C 4...
  requests: <list(2)> query, imageinfo
  title: Cassiopeia A
  url: https://en.wikipedia.org/wiki/Cassiopeia_A
  url_raw: https://en.wikipedia.org/wiki/Cassiopeia_A?action=raw
  watchers: 37
  wikibase: Q618273
  wikidata_url: https://www.wikidata.org/wiki/Q618273
}
en.wikiped

RAstring 23<sup>h</sup> 23<sup>m</sup> 24<sup>s</sup>
RA 23<sup>h</sup> 23<sup>m</sup> 24<sup>s</sup 23<sup>h</sup> 23<sup>m</sup> 24<sup>s</sup 23<sup>h</sup> 23<sup>m</sup> 24<sup>s</sup


Cassiopeia A (en) data
{
  WARNINGS: <dict(1)> extracts
  assessments: <dict(1)> Astronomy
  description: supernova remnant in the constellation Cassiopeia
  extext: <str(1658)> **Cassiopeia A** ( **Cas A** ) is a supernov...
  extract: <str(1714)> <p><b>Cassiopeia A</b> (<b>Cas A</b>) is a ...
  image: <list(2)> {'kind': 'query-pageimage', 'file': 'File:Cassi...
  infobox: <dict(20)> name, image, caption, type, epoch, SNRtype, ...
  iwlinks: <list(3)> https://commons.wikimedia.org/wiki/Category:C...
  label: Cassiopeia A
  length: 14,046
  links: <list(140)> 3 Cassiopeiae, Aerobee, All Sky Automated Sur...
  modified: <dict(1)> page
  pageid: 1996903
  parsetree: <str(18842)> <root><template><title>Infobox supernova...
  random: Burin-Placentia West
  redirects: <list(6)> {'pageid': 1499533, 'ns': 0, 'title': '3C 4...
  requests: <list(3)> query, imageinfo, parse
  title: Cassiopeia A
  url: https://en.wikipedia.org/wiki/Cassiopeia_A
  url_raw: https://en.wikipedia.org/wiki/Cassiopeia

ValueError: could not convert string to float: '23<sup>h</sup> 23<sup>m</sup> 24<sup>s</sup'

### Compile all the objects for the interactive

In [ ]:
import compileObjects
compileObjects